### ライブラリのインポート

In [25]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import os
import math
import optuna
import re
import random

### 計算用関数

In [26]:
# 行数を指定すると所用時間を返す関数
def knapsack(row_x, data_x,required_time_x, waiting_time_x, move_time_x, attraction_no_x):
    waiting_time_x = data_x.iloc[row_x, attraction_no_x]
    ans = required_time_x + waiting_time_x + move_time_x
    return ans


# 合計所要時間と合計人気度を算出する関数
def ride_calculation(data, ride_lst, required_times, popularity, move_time):
    # print(data)
    # print(ride_lst)
    # print(required_times)
    # print(move_time)
    total_popularity = 0
    total_time = 0
    ride_time = []
    new_ride_lst = []
    # start_time = data.index[0]
    # print("start time is:", calculate_time(start_time))
    row = 0
    try:
        c=0
        for i in ride_lst:
            # total_timeが675以上かつiが0または1の場合にループを強制終了
            # ソアリン、トイストーリーMは20:15以降DPA所持者のみ乗車可能
            if (data.index[row]) >= 675 and (i == 0 or i == 1):
                c+=1
                continue
            
            else:
                # iが8以上の場合、次のループへ(該当しないアトラクション)
                if i >= 8 or i < 0:
                    # print("不明なアトラクション：", i)
                    c+=1
                    continue
                elif c>0 and i==ride_lst[c-1]:
                    c+=1
                    continue
                else:
                    # print("-" * 40)
                    # print(calculate_time(data.index[row]))
                    ride_time.append(data.index[row])
                    # print(attractions.get(i))
                    # print("移動時間", move_time)
                    waiting_time = data.iloc[row, i]
                    # print("待ち時間:", waiting_time)
                    required_time = required_times[i]
                    # print("観賞時間:", required_time)
                    ans = knapsack(row, data, required_time, waiting_time, move_time, i)
                    # print("合計所要時間:", ans)
                    total_time += ans
                    row = int(np.ceil(total_time / 15))
                    total_popularity += popularity[i]
                    new_ride_lst.append(i)
                    c+=1
    except Exception as e:
        # print("全てのアトラクションを乗ることができません")
        pass

    # print("-" * 40)
    # print(total_popularity)
    return total_popularity, total_time, new_ride_lst, ride_time

### 最適化実行関数

In [27]:
#def constraints(trial):
#    return trial.user_attrs["constraints"]

def optimization(file_pathes):
    best_plan_lst = []
    file_path = file_pathes

    # 目的関数の定義
    def objective(trial):
        # アトラクションの所要時間と人気度 or レビュー評価
        required_times = [5, 7, 2, 3, 3, 2, 23, 30]  # 所要時間
        popularity = [476, 465, 473, 472, 480, 452, 461, 478] # レビュー評価
        #popularity = [84, 69, 55, 45, 33, 28, 19, 22]  # 人気度
        move_time = 15
        
        # print(file_path)
        data = pd.read_csv(file_path)
        data = data.set_index('時間')
        
        # 乗り物の数を決定
        n_rides = trial.suggest_int('n_rides', 8, 20)  # 最大20の乗り物を仮定
        
        
        # まず0~7をすべて含む部分を作る
        #ride_lst = list(range(8))
    
        # もしn_ridesが8を超える場合、残りの要素をランダムに追加
        #if n_rides > 8:
        #    ride_lst += [trial.suggest_int(f'ride_{i}', 0, 7) for i in range(n_rides - 8)]
    
        # リストをシャッフルして順番をランダムに
        #random.shuffle(ride_lst)        
                
        
        # 乗り物の定義
        ride_lst = [trial.suggest_int(f'ride_{i}', 0, 7) for i in range(n_rides)]
        
        # ride_calculation関数の呼び出し
        total_popularity, total_time, new_ride_lst, ride_time = ride_calculation(data, ride_lst, required_times, popularity, move_time)
        
        # 制約条件の範囲
        #max_allowed_time = 660  # パーク内にいる時間 9:00~20:00 計660分 (20時以降はファストパスのみ受け入れと仮定)
        max_allowed_time = 675  # パーク内にいる時間 9:00~20:15 計675分 (20:15以降はファストパスのみ受け入れと仮定)
        
        # 制約を満たす場合のみ最適化対象とする
        if total_time <= max_allowed_time:
            # 正規表現で年月日部分を抽出
            match = re.search(r'\d{4}-\d{2}-\d{2}', file_path)
            date = match.group() if match else None
            
            best_plan_lst.append([trial.number, date, total_popularity, total_time, ride_lst, new_ride_lst, ride_time])
            # print(total_popularity)
            return total_popularity
        else:
            # 制約を満たさない場合はペナルティを課す
            return float('-inf')
        
        
        #####
        # 制約を満たす場合のみ最適化対象とする
        #trial.set_user_attr("constraints",[max_allowed_time - total_time])
        # 正規表現で年月日部分を抽出
        #match = re.search(r'\d{4}-\d{2}-\d{2}', file_path)
        #date = match.group() if match else None
        #best_plan_lst.append([trial.number, date, total_popularity, total_time, ride_lst, new_ride_lst, ride_time])
        # print(total_popularity)
        #return total_popularity
        #####
            
        

    # Optunaによる最適化
    study = optuna.create_study(direction="maximize")  # 人気度を最大化
    study.optimize(objective, n_trials = 300)

    # 結果の表示
    best_trial = study.best_trial
    print(f"Best trial: {best_trial.number}")
    print(f"Best popularity: {best_trial.value}")
    print(f"Parameters: {best_trial.params}")
    
    # DataFrameに変換
    best_plan_df = pd.DataFrame(best_plan_lst)
    
    # best_trial.numberに対応する行を抽出
    best_plan = best_plan_df[best_plan_df[0] == best_trial.number].iloc[0]
    
    return best_plan

In [1]:
columns = ["日付", "トータル満足度", "トータル所要時間", "予定プラン", "最適プラン", "タイムスケジュール"]
df_best_csv = pd.DataFrame(columns=columns)


import time
# 処理時間の計算
start_time = time.time()

# 最適化の実行
folder_path = 'date_data2/' ←
#folder_path = '0819/'
best_csv = []
file_names = sorted(os.listdir(folder_path))  # フォルダ名をソート
for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    if file_path.endswith('.csv'):
        for i in range(10): # ファイルにつき何回最適化を行うか
            plan = optimization(file_path)
            plan_data = plan.iloc[1:]
            best_csv.append(plan_data.values.tolist())
            # You can add additional logic to handle the results of each optimization run here

# best_csvリストをDataFrameに変換
columns = ["日付", "トータル満足度", "トータル所要時間", "予定プラン", "最適プラン", "タイムスケジュール"]
df_best_csv = pd.DataFrame(best_csv, columns=columns)

# 日付をインデックスに設定
df_best_csv = df_best_csv.set_index("日付")

# DataFrameをCSVファイルとして保存
#df_best_csv.to_csv('best_results_230819_r100_t1500_review.csv', index=True)
df_best_csv.to_csv('best_results_alldays_r10_t300_review.csv', index=True)



end_time = time.time()
# 処理時間を計算
elapsed_time = end_time - start_time
# 処理時間を表示
print(f"処理時間: {elapsed_time} 秒")

SyntaxError: invalid character '←' (U+2190) (1521583141.py, line 10)

In [29]:
print(f"処理時間: {elapsed_time} 秒")

処理時間: 101082.93539977074 秒


In [30]:
df = pd.read_csv("best_results_alldays_r10_t300_review.csv")
display(df)

,日付,トータル満足度,トータル所要時間,予定プラン,最適プラン,タイムスケジュール
0,2022-07-01,8028,663,"[3, 4, 3, 1, 0, 2, 3, 4, 3, 4, 3, 5, 3, 2, 3, ...","[3, 4, 3, 1, 0, 2, 3, 4, 3, 4, 3, 5, 3, 2, 3, ...","[0, 45, 75, 135, 195, 255, 315, 345, 390, 420,..."
1,2022-07-01,8050,673,"[7, 3, 2, 4, 0, 4, 7, 3, 4, 3, 5, 5, 4, 2, 4, ...","[7, 3, 2, 4, 0, 4, 7, 3, 4, 3, 5, 4, 2, 4, 3, ...","[0, 45, 120, 180, 195, 255, 300, 360, 405, 420..."
2,2022-07-01,8030,657,"[4, 5, 7, 2, 4, 3, 5, 4, 0, 4, 3, 3, 4, 3, 4, ...","[4, 5, 7, 2, 4, 3, 5, 4, 0, 4, 3, 4, 3, 4, 5, ...","[0, 30, 60, 120, 180, 195, 240, 270, 300, 360,..."
3,2022-07-01,7124,670,"[6, 7, 4, 3, 0, 4, 0, 7, 4, 3, 4, 2, 1, 4, 2, 0]","[6, 7, 4, 3, 0, 4, 0, 7, 4, 3, 4, 2, 1, 4, 2]","[0, 45, 90, 135, 180, 255, 300, 360, 435, 450,..."
4,2022-07-01,7914,639,"[4, 6, 5, 1, 3, 5, 5, 2, 6, 3, 3, 6, 4, 3, 5, ...","[4, 6, 5, 1, 3, 5, 2, 6, 3, 6, 4, 3, 5, 1, 3, ...","[0, 30, 75, 105, 165, 225, 255, 315, 360, 405,..."
...,...,...,...,...,...,...
1175,2023-08-30,5205,660,"[4, 6, 4, 6, 4, 3, 4, 3, 6, 7, 4]","[4, 6, 4, 6, 4, 3, 4, 3, 6, 7, 4]","[0, 30, 75, 150, 225, 270, 345, 420, 480, 540,..."
1176,2023-08-30,5155,651,"[5, 4, 7, 5, 6, 7, 3, 7, 5, 3, 3, 4, 4]","[5, 4, 7, 5, 6, 7, 3, 7, 5, 3, 4]","[0, 30, 90, 165, 225, 270, 345, 420, 480, 525,..."
1177,2023-08-30,5617,673,"[5, 6, 7, 6, 2, 4, 4, 5, 4, 7, 3, 3, 7, 5]","[5, 6, 7, 6, 2, 4, 5, 4, 7, 3, 7, 5]","[0, 30, 60, 135, 195, 270, 345, 405, 465, 525,..."
1178,2023-08-30,4734,659,"[2, 4, 1, 2, 3, 4, 6, 7, 3, 4]","[2, 4, 1, 2, 3, 4, 6, 7, 3, 4]","[0, 60, 135, 225, 285, 360, 420, 480, 540, 615]"
